#Задание

**Задача**

1) Создать синтетический набор данных используя открытую LLM для решения поставленной задачи

2) Обучить модель классификации текста на основе созданного набора данных используя трансформерную модель энкодер (например BERT)

**Генерация данных**

Требования к данным:  

1)Набор даных должен представлять диалоги клиентов с поддержкой на тему банковских услуг (кредиты, денжные переводы и т.д).

2)Набор данных должен быть создан для решения задачи классификации последнего сообщения пользователя в диалоге - (класс - текущий вопрос пользователя по которому будет выдаваться некоторые заранее заготовленный ответ)

3)В Наборе данных должно быть не менее 15 классов (больше можно), выполняющий имеет свободу выбора сам составить предполагаемые классы сообщений или же сгенерировать их. Для каждого класса нужно сгенерировать не менее 20 различны примеров для обучения и 10 для теста.

4) Формат данных:

{"label": 'transfer_status',

"dialog":

"user: Привет! можете помочь мне ?

assistant: Здравствуйте! В чем ваш вопрос?

user: Мой перевод все еше не пришел, какой у него cтатус ?"}

5) Напоминание, класс определяется именно для последнего сообщения в контексте диалога, а не для последнего сообщения в отдельности или же диалога в целом.

**Что поможет**

1) Все модели и гайды искать на https://huggingface.co/

2) Модель можно запустить как через transformers, так и через специализированные движки (приветствуется ) llama.cpp / VLLM

3)Ресурсов Colab или же Kaggle достаточно для генерации за 1 итерацию.

**Обучение классификатора**

1) Достаточный вариант - реализовать обучение через Trainer из huggingface transformers, на портале есть множество гайдов в том числе для решения задачи классификации текста с помощью моделей энкодеров (например BERT)

2)При желании можно показать свои навыки в написании кастомных циклов обучения, не используя готовый класс Trainer

3)Желательно использовать transformers + pytorch, другие варианты (например через tensorflow также будут проверяться, но этот фреймворк редко используется в практике проверяющих)

**Как предсатвить результат ?**

Jupyter notebook, желательно один с обоими частями задачи и обязательно содержащий наглядную демонстрацию результатов - примеры сгенерированных диалогов для каждого класса, обучающие и тестовые метрики оубчения классификатора.

#Установка и импорт зависимостей

In [1]:
!pip install transformers accelerate torch tqdm datasets evaluate accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00


In [9]:
from transformers import pipeline
from tqdm import tqdm
import json
import random

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict

# Синтез данных

In [2]:
model_name = "microsoft/phi-3-mini-4k-instruct"

generator = pipeline(
    "text-generation",
    model=model_name,
    torch_dtype="auto",
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
classes = [
    "transfer_status",
    "card_block",
    "loan_request",
    "account_opening",
    "loan_debt",
    "transaction_cancel",
    "lost_card_report",
    "interest_rate_info",
    "installment_payment",
    "foreign_transfer",
    "atm_issue",
    "mobile_app",
    "fraud_alert",
    "balance_check",
    "currency_exchange_rate"
]

ru_classes = [
    "статус перевода",
    "блокировка карты",
    "запрос кредита",
    "открытие счёта",
    "кредитная задолженность",
    "отмена транзакции",
    "потеря карты",
    "процентная ставка",
    "рассрочка",
    "зарубежный перевод",
    "неисправность банкомата",
    "мобильное приложение",
    "мошенничество",
    "проверка баланса",
    "курс валют"
]



## Создание наборов изначальных реплик для пользователя и ассистента

In [16]:
prompt = f"""
Ты — пользователь банка, которому нужна помощь от банковского ассистента.
Сгенерируй 6 различных приветствий от пользователя к банковскому ассистенту, без уточнения проблемы.
Каждое приветствие разделить переносом строки.
"""

response = generator(prompt, max_new_tokens=200, temperature=0.7, top_p=0.8, do_sample=True)
print(response[0]["generated_text"])


Ты — пользователь банка, которому нужна помощь от банковского ассистента.
Сгенерируй 6 различных приветствий от пользователя к банковскому ассистенту, без уточнения проблемы. 
Каждое приветствие разделить переносом строки. 

Привет, я хотел бы обсудить мои вопросы о банковских услугах.
Здравствуйте, я захотел бы поговорить о моих интересах в банковских продуктах.
Здравствуйте, я хочу поговорить о разнообразных услугах банка, которые могли бы удовлетворить моему потребность.
Приветствую, я хочу обсудить мои заинтересованности в банковских услугах.
Здравствуйте, я хочу узнать больше о вашем банке и его предложениях.
Приветствую, я хочу обсудить мои вопросы и попросить вашего совета.

## Response:Привет


In [17]:
with open('my_file.txt', 'w', encoding='utf-8') as file:
    file.write(response[0]["generated_text"])

In [12]:
prompt1 = f"""
Ты — банковский ассистент-помощник.
Сгенерируй 6 разных приветствий от ассистента к пользователю банка, которому нужна помощь.
Каждое приветствие разделить переносом строки.
"""

response1 = generator(prompt1, max_new_tokens=500, temperature=0.7, top_p=0.8, do_sample=True)
print(response1[0]["generated_text"])


Ты — банковский ассистент-помощник.
Сгенерируй 6 разных приветствий от ассистента к пользователю банка, которому нужна помощь. 
Каждое приветствие разделить переносом строки.

#### 1. Привет, [[Пользователь]]! Мы готовы помочь вам с вопросами о банке. Как можем быть вашими помощниками?

#### 2. Добрый день, [[Пользователь]]! Мы на связи с вами, [[Банк]]. Чтобы отобразить ваш счет, просто отправьте команду: "show balance".

#### 3. Приветствую [[Пользователь]], [[Банк]]. Мы в курсе ваших запросов и готовы предоставить вам необходимую помощь.

#### 4. Здравствуйте, [[Пользователь]], [[Банк]]. Мы на связи с вами, и все ваши вопросы о банке мы предоставляем.

#### 5. Привет, [[Пользователь]]! Мы [[Банк]] и всегда готовы помочь вам с любыми вопросами.

#### 6. Привет [[Пользователь]], [[Банк]]. Мы всегда готовы помочь вам с любыми вопросами о банке.


In [18]:
with open('my_file1.txt', 'w', encoding='utf-8') as file:
    file.write(response1[0]["generated_text"])

## Создание вопросов пользователя ассистенту по классам

In [7]:
def generate_question(label, isexample):
  prompt2 = f"""
Ты — пользователь банка, у которого возникла проблема по теме "{label}".
Сгенерируй 30 различных вопросов по теме "{label}".
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!
"""

  response2 = generator(prompt2, max_new_tokens=2000, temperature=0.7, top_p=0.8, do_sample=True)
  if isexample == True:
    print(response2[0]["generated_text"])

  return response2[0]["generated_text"]


In [8]:
for i in tqdm(range(15)):

  isexample = False
  if i == 0:
    isexample = True

  filename = f"{classes[i]}.txt"
  gendata = generate_question(ru_classes[i], isexample)

  with open(filename, 'w', encoding='utf-8') as file:
    file.write(gendata)

  7%|▋         | 1/15 [00:49<11:26, 49.01s/it]


Ты — пользователь банка, у которого возникла проблема по теме "статус перевода".
Сгенерируй 30 различных вопросов по теме "статус перевода".
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!

### Answer:1. Как долго надо подождать после подачи заявки о статус перевода?
2. Какие документы необходимо прикрепить к заявке о статус перевода?
3. Если статус перевода оказывается неожиданно неудовлетворительным, что следует делать?
4. Как можно узнать, когда статус перевода будет установлен?
5. Что делать, если банка не отвечает на запрос о статус перевода?
6. Можно ли отменить статус перевода, если он был предоставлен с ошибками?
7. Как долго банк может откладывать статус перевода после проверки документов?
8. Как можно увидеть статус перевода в электронном режиме?
9. Если статус перевода оказывается неудовлетворительным, возможно ли отправить новую заявку?
10. Какие условия банка могут привести к отказу в статус перевода?
11. Можно ли уведомить банку 

100%|██████████| 15/15 [14:32<00:00, 58.14s/it]


## Доработка классов, чьи данные были неудачно сгенерированы

Мобильное приложение

In [5]:
promptma = f"""
Ты — обычный пользователь, у которого возникла проблема в мобильном приложениии банка.
Сгенерируй 30 различных вопросов, которые могут возникнуть у пользователя при использовании мобильного приложения банка.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!
"""

responsema = generator(promptma, max_new_tokens=2000, temperature=0.7, top_p=0.8, do_sample=True)
print(responsema[0]["generated_text"])


Ты — обычный пользователь, у которого возникла проблема в мобильном приложениии банка.
Сгенерируй 30 различных вопросов, которые могут возникнуть у пользователя при использовании мобильного приложения банка.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!

1. Привет, я не могу войти в мой банк через мобильное приложение, что делать?
2. Могу ли я отменить перевод с мобильного приложения банка, если я ошибся?
3. Как я могу проверить, что мой счет обновлен после сделки?
4. Могу ли я выбрать другой банк для сделок через мобильное приложение?
5. Как я могу отправить сообщение банку по сделкам через мобильное приложение?
6. Могу ли я установить расширения для мобильного приложения банка?
7. Как я могу проверить историю транзакций в мобильном приложении банка?
8. Могу ли я использовать мобильное приложение банка на планшете или только на мобильном телефоне?
9. Как я могу установить банковский карту на мобильно

In [6]:
with open('mobile_app.txt', 'w', encoding='utf-8') as file:
  file.write(responsema[0]["generated_text"])

Потеря карты

In [10]:
promptlsr = f"""
Ты — пользователь банка, который потерял банковскую карту.
Сгенерируй 30 различных вопросов от лица пользователя, связанные с утерей банковской карты, которые пользователь может задать банку.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен должен отличаться от предыдущего и должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!
"""

responselsr = generator(promptlsr, max_new_tokens=2000, temperature=0.7, top_p=0.8, do_sample=True)
print(responselsr[0]["generated_text"])


Ты — пользователь банка, который потерял банковскую карту.
Сгенерируй 30 различных вопросов от лица пользователя, связанные с утерей банковской карты, котроые пользователь может задать банку.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен должен отличаться от предыдущего и должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!


### Answer:1. Привет, я заметил, что моя банковская карта потерялась. Могу ли я попросить ее возвратить?
2. Мне очень важно, что я не оставлю свой банковский счет без карты. Могу ли я связаться с вами по этой проблеме?
3. Мне сказали, что моя банковская карта украдена. Могу ли я сделать заявку на ее возврат?
4. Я учусь в университете, и я не могу сейчас использовать банковскую карту. Могу ли я запросить ее возврат?
5. Мне нужно отправить платеж на другую страну, но у меня нет карты. Могу ли я связаться с вами по этой проблеме?
6. Мне не удалось найти свою банковскую карту после переезда. Могу ли я сделать запрос на ее возвра

In [13]:
responselsr2 = generator(promptlsr, max_new_tokens=2000, temperature=0.7, top_p=0.8, do_sample=True)
print(responselsr2[0]["generated_text"])


Ты — пользователь банка, который потерял банковскую карту.
Сгенерируй 30 различных вопросов от лица пользователя, связанные с утерей банковской карты, котроые пользователь может задать банку.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен должен отличаться от предыдущего и должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!

**Solution:1**

1. Привет, я потерял свою банковскую карту, может помочь?
2. Как я могу узнать, была ли моя карта украдена?
3. Я не могу найти мою банковскую карту, что делать?
4. Могу ли я заменить утерянную карту?
5. Как быстро я могу сбросить карту?
6. Могу ли я сбросить карту без переподписки на новую?
7. Я хочу убедиться, что карта не украдена, как это сделать?
8. Как я могу проверить, не украден ли моя карта?
9. Как быстро можно сделать новую карту?
10. Могу ли я сбросить карту в другой банк?
11. Как я могу убедиться, что карта не украдена, если она не отображается в банке?
12. Могу ли я сбросить карту, если я не уверен

In [14]:
with open('lost_card_report.txt', 'w', encoding='utf-8') as file:
  file.write(responselsr[0]["generated_text"])
  file.write(responselsr2[0]["generated_text"])

Обмен валюты

In [15]:
promptcr = f"""
Ты — пользователь банка, которому нужно обменять валюту.
Сгенерируй 30 различных вопросов, которые связаны с курсом и обменом валют.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!
"""

responsecr = generator(promptcr, max_new_tokens=2000, temperature=0.7, top_p=0.8, do_sample=True)
print(responsecr[0]["generated_text"])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Ты — пользователь банка, которому нужно обменять валюту.
Сгенерируй 30 различных вопросов, которые связаны с курсом и обменом валют.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!

### Support:1. Привет! Меня зовут Анна, и я хочу обменять доллары на рубли. Какова валюта сейчас?
2. Какова сумма рублей, которую я могу получить при обмене 100 долларов?
3. Могу я обменять рубли на евро?
4. Как долго я могу ожидать, чтобы полученные рубли заработали?
5. Могу ли я обменять рубли на собаки?
6. Какова цена одного собаки в рублях?
7. Как я могу обменять рубли на валюту, если она не поддерживается банком?
8. Могу ли я обменять рубли на билеты на самолеты?
9. Какова сумма рублей, которую я могу получить при обмене 500 долларов?
10. Могу я обменять рубли на билеты на поезд?
11. Какова цена одного билета на поезд в рублях?
12. Могу ли я обменять рубли на автомобили?
13. Какова цена одного автомобиля в рублях?
14. М

In [17]:
with open('currency_exchange_rate.txt', 'w', encoding='utf-8') as file:
  file.write(responsecr[0]["generated_text"])

Кредитная задолженность

In [18]:
promptld = f"""
Ты — пользователь банка, у которого возникли проблемы с задолженностью по кредиту.
Сгенерируй 30 различных вопросов, которые связаны с задолженностью и погашением кредита.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!
"""

responseld = generator(promptld, max_new_tokens=2000, temperature=0.7, top_p=0.8, do_sample=True)
print(responseld[0]["generated_text"])


Ты — пользователь банка, у которого возникли проблемы с задолженностью по кредиту.
Сгенерируй 30 различных вопросов, которые связаны с задолженностью и погашением кредита.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!

Пример вопроса:

- Привет, я не могу понять, как мне помочь с моим кредитом. Я должен каким-то образом увеличить свои доходы?

- У меня нет доступа к банковскому интерфейсу. Как мне устроить счет на погашение кредита?

- Я учусь, и у меня нет дохода. Могу ли я наложить срочное платеж на кредит?

- Я потерял свой рабочий график, и теперь не могу вернуть кредит. Что мне делать?

- Я очень устал от неудачных попыток погашения кредита. Может быть, банк может сделать какую-то гарантию на мой счет?

- Мне нужно отменить кредит, потому что я не могу его погашать. Как мне это сделать?

- Я не могу найти информацию о том, как банк может помочь мне с уплатой кредита. Пожалуйста, дайте мне помощь!

In [19]:
responseld2 = generator(promptld, max_new_tokens=2000, temperature=0.7, top_p=0.8, do_sample=True)
print(responseld2[0]["generated_text"])


Ты — пользователь банка, у которого возникли проблемы с задолженностью по кредиту.
Сгенерируй 30 различных вопросов, которые связаны с задолженностью и погашением кредита.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!

1. Как я могу узнать, сколько осталось на моём кредите?
2. Могу ли я снять кредит за дополнительный период?
3. Могу ли я переезднуть и не возвращаться на свой прежний адрес?
4. Могу ли я сменить банк на свой собственный?
5. Как я могу предотвратить отток банка?
6. Могу ли я сменить банк на другой?
7. Как я могу уменьшить сумму погашения кредита?
8. Могу ли я погасить кредит в несколько частей?
9. Могу ли я снять кредит без предоплаты?
10. Как я могу уменьшить процент погашения кредита?
11. Могу ли я снять кредит на иностранном счете?
12. Могу ли я снять кредит на недвижимость?
13. Как я могу уменьшить процент на кредит?
14. Могу ли я снять кредит на автомобиль?
15. Как я могу уменьшить 

In [20]:
with open('loan_debt.txt', 'w', encoding='utf-8') as file:
  file.write(responseld[0]["generated_text"])
  file.write(responseld2[0]["generated_text"])

Блокировка карты

In [22]:
promptcb = f"""
Ты — пользователь банка, у которого возникли проблемы с блокировкой карты.
Сгенерируй 30 различных вопросов, которые связаны с блокировкой и разблокировкой карты.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!
"""

responsecb = generator(promptcb, max_new_tokens=2000, temperature=0.7, top_p=0.8, do_sample=True)
print(responsecb[0]["generated_text"])


Ты — пользователь банка, у которого возникли проблемы с блокировкой карты.
Сгенерируй 30 различных вопросов, которые связаны с блокировкой и разблокировкой карты.
Напиши эти вопросы от лица пользователя!
Каждый вопрос должен быть разделён переносом строки.
Ответы на вопросы выводить не нужно!


## Response:1. Мой банк мне сказал, что моя карта была блокирована. Что это может означать?
2. Мне было заблокировано моя карта, почему это может произойти?
3. Какие могут быть причины блокировки карты?
4. Могу ли я повторно попросить банк разблокировать мою карту?
5. Какие шаги мне следует предпринять, чтобы разблокировать мою карту?
6. Могу ли я откатить блокировку карты, если я ошибся в оплату?
7. Как долго банк может блокировать мою карту?
8. Могу ли я вызвать банк, чтобы объяснить ситуацию и разблокировать мою карту?
9. Могу ли я сделать оплату через другой канал, чтобы разблокировать мою карту?
10. Могу ли я заблокировать свою карту самостоятельно?
11. Какие могут быть последствия блокиро

In [23]:
with open('card_block.txt', 'w', encoding='utf-8') as file:
  file.write(responsecb[0]["generated_text"])

# Оформление сгенерированных данных

Тексты, которые я загрузила в файлы, я очистила вручную, убрав пояснения и повторяющиеся предложения

In [12]:
classes = [
    "transfer_status",
    "card_block",
    "loan_request",
    "account_opening",
    "loan_debt",
    "transaction_cancel",
    "lost_card_report",
    "interest_rate_info",
    "installment_payment",
    "foreign_transfer",
    "atm_issue",
    "mobile_app",
    "fraud_alert",
    "balance_check",
    "currency_exchange_rate"
]


Полученные предложения я добавила во временный словарь, чтобы потом можно было удобно составлять диалоги

In [13]:
gendata = {}
gendata["ugreet"] = []
gendata["agreet"] = []

with open('my_file.txt', 'r', encoding='utf-8') as file:
  for line in file:
    if line != '':
      gendata["ugreet"].append(line.rstrip('\n'))

with open('my_file1.txt', 'r', encoding='utf-8') as file:
  for line in file:
    if line != '':
      gendata["agreet"].append(line.rstrip('\n'))

for bank_class in tqdm(classes):
  gendata[bank_class] = []
  filename = f'{bank_class}.txt'
  with open(filename, 'r', encoding='utf-8') as file:
    for line in file:
      if line != '' and line not in gendata[bank_class]:
        gendata[bank_class].append(line.rstrip('\n'))

  random.shuffle(gendata[bank_class])


100%|██████████| 15/15 [00:00<00:00, 6125.46it/s]


Диалоги я составляла по следующему принципу: Берём случайное привестствие пользователя -> Берём случайное приветствие ассистента -> Вопрос пользователя по классу

В некоторых классах уникальных вопросов было сгенерировано меньше, чем 30, но так как у нас есть 36 комбинаций приветствий пользователя и ассистента, то диалоги будут уникальными

Примеры диалогов представлены в выводе

In [16]:
bert_data = {}
bert_data['train'] = []
bert_data['test'] = []

for bank_class in tqdm(classes):

  for i in range(20):
    new_entry = {}
    new_entry['label'] = bank_class

    ugreet = random.choice(gendata["ugreet"])
    agreet = random.choice(gendata["agreet"])

    uq_index = i % len(gendata[bank_class])
    uq = gendata[bank_class][uq_index]

    new_entry['dialog'] = f'user: {ugreet} assistant: {agreet} user: {uq}'
    bert_data['train'].append(new_entry)

    if i==0:
      print(new_entry)

  for i in range (20, 30):
    new_entry = {}
    new_entry['label'] = bank_class

    ugreet = random.choice(gendata["ugreet"])
    agreet = random.choice(gendata["agreet"])

    uq_index = i % len(gendata[bank_class])
    uq = gendata[bank_class][uq_index]

    new_entry['dialog'] = f'user: {ugreet} assistant: {agreet} user: {uq}'
    bert_data['test'].append(new_entry)

random.shuffle(bert_data['train'])
random.shuffle(bert_data['test'])

100%|██████████| 15/15 [00:00<00:00, 6249.58it/s]

{'label': 'transfer_status', 'dialog': 'user: Приветствую, я хочу обсудить мои заинтересованности в банковских услугах. assistant: Привет, [[Пользователь]]! Мы готовы помочь вам с вопросами о банке. Как можем быть вашими помощниками? user: Могут ли банки устанавливать статус перевода без возможности отмены?'}
{'label': 'card_block', 'dialog': 'user: Приветствую, я хочу обсудить мои вопросы и попросить вашего совета. assistant: Привет, [[Пользователь]]! Мы [[Банк]] и всегда готовы помочь вам с любыми вопросами. user: Могу ли я вызвать банк, чтобы объяснить ситуацию и разблокировать мою карту?'}
{'label': 'loan_request', 'dialog': 'user: Привет, я хотел бы обсудить мои вопросы о банковских услугах. assistant: Приветствую [[Пользователь]], [[Банк]]. Мы в курсе ваших запросов и готовы предоставить вам необходимую помощь. user: Какие данные требуются для подачи запроса кредита?'}
{'label': 'account_opening', 'dialog': 'user: Приветствую, я хочу обсудить мои заинтересованности в банковских у

In [19]:
json_bert_data = json.dumps(bert_data, ensure_ascii=False)

with open('bert_data.json', 'w', encoding='utf-8') as file:
  file.write(json_bert_data)


# Обучение модели-энкодера

In [4]:
with open('bert_data.json', 'r') as file:
  bert_data = json.load(file)

In [18]:
train_dataset = Dataset.from_list(bert_data['train'])
test_dataset = Dataset.from_list(bert_data['test'])

bert_dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
def preprocess_function(examples):
    return tokenizer(examples["dialog"], truncation=True)

In [11]:
tokenized_train_data = bert_dataset["train"].map(preprocess_function, batched=True)
tokenized_test_data = bert_dataset["test"].map(preprocess_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)

    acc = accuracy.compute(predictions=preds, references=labels)
    prec = precision.compute(predictions=preds, references=labels, average="weighted")
    rec = recall.compute(predictions=preds, references=labels, average="weighted")
    f1_score = f1.compute(predictions=preds, references=labels, average="weighted")

    return {
        "accuracy": acc["accuracy"],
        "precision": prec["precision"],
        "recall": rec["recall"],
        "f1": f1_score["f1"],
    }

In [17]:
id2label = {}
label2id = {}

classes = [
    "transfer_status",
    "card_block",
    "loan_request",
    "account_opening",
    "loan_debt",
    "transaction_cancel",
    "lost_card_report",
    "interest_rate_info",
    "installment_payment",
    "foreign_transfer",
    "atm_issue",
    "mobile_app",
    "fraud_alert",
    "balance_check",
    "currency_exchange_rate"
]

for i in range(15):
  id2label[i] = classes[i]
  label2id[classes[i]] = i

id2label, label2id

({0: 'transfer_status',
  1: 'card_block',
  2: 'loan_request',
  3: 'account_opening',
  4: 'loan_debt',
  5: 'transaction_cancel',
  6: 'lost_card_report',
  7: 'interest_rate_info',
  8: 'installment_payment',
  9: 'foreign_transfer',
  10: 'atm_issue',
  11: 'mobile_app',
  12: 'fraud_alert',
  13: 'balance_check',
  14: 'currency_exchange_rate'},
 {'transfer_status': 0,
  'card_block': 1,
  'loan_request': 2,
  'account_opening': 3,
  'loan_debt': 4,
  'transaction_cancel': 5,
  'lost_card_report': 6,
  'interest_rate_info': 7,
  'installment_payment': 8,
  'foreign_transfer': 9,
  'atm_issue': 10,
  'mobile_app': 11,
  'fraud_alert': 12,
  'balance_check': 13,
  'currency_exchange_rate': 14})

In [19]:
def encode_labels(example):
    example["label"] = label2id[example["label"]]
    return example

tokenized_train_data = tokenized_train_data.map(encode_labels)
tokenized_test_data = tokenized_test_data.map(encode_labels)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-multilingual-uncased", num_labels=15, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
training_args = TrainingArguments(
    output_dir="shift_model",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.591902,0.100000,0.062648,0.100000,0.054487
2,No log,2.233084,0.426667,0.512213,0.426667,0.368188


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=150, training_loss=2.5612221272786457, metrics={'train_runtime': 110.6075, 'train_samples_per_second': 5.425, 'train_steps_per_second': 1.356, 'total_flos': 33221647975680.0, 'train_loss': 2.5612221272786457, 'epoch': 2.0})

Результаты работы модели

In [21]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 2.233084201812744, 'eval_accuracy': 0.4266666666666667, 'eval_precision': 0.5122125435540069, 'eval_recall': 0.4266666666666667, 'eval_f1': 0.36818767289355525, 'eval_runtime': 1.443, 'eval_samples_per_second': 103.953, 'eval_steps_per_second': 26.335, 'epoch': 2.0}


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
with open("results.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=2, ensure_ascii=False)